In [9]:
import os
import time
import datetime

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.nn.parallel import DataParallel
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, T5Config, T5ForConditionalGeneration
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

import warnings
warnings.filterwarnings("ignore")

import nltk
# nltk.download('punkt')
# 类


In [10]:
path = '/home/hkqiu/work/PolyGPT/Polymer-Generation/model_save/PI1M_pretrain'
tokenizer = AutoTokenizer.from_pretrained(path, pad_token='[PAD]', padding_side='right') 
configuration = T5Config.from_pretrained(path, output_hidden_states=False)
model = T5ForConditionalGeneration.from_pretrained(path, config=configuration)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32114, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32114, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [11]:
prompt = "201,15,1,1,1,1,2,0,0,0,1,1,3,3,2"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generated = generated.to(device)  # 在GPU上生成会快一倍

# sample_outputs = model.module.generate(
sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   # 是否采样，True表示采样搜索，可以设置返回结果数目。False表示贪婪搜索，只能返回一个结果
                                top_k=100, 
                                max_length = 300,
                                top_p=0.999, 
                                num_return_sequences=5
                                )

for i, sample_output in enumerate(sample_outputs):
    print("{}: {}\n\n".format(i+1, tokenizer.decode(sample_output, skip_special_tokens=True)))

1: <pad> *NC1CCC(C=C2CCCCCC(*)CC2)C1<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


2: <pad> *CC(C)(C)C1C(=*)CC2CCCC(CO)C2C1<pad><pad><pad><pad><pad>


3: <pad> *CC(*)C1=CC=C2C=CC=C(C=C(C)N)C2=C1


4: <pad> *NC1(CC(C)(C)CC2C(C)CC(*)C2C)C1<pad><pad><pad><pad>


5: <pad> *CCCC1CCCC(*)C(C)(CN)C1<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


